In [102]:
from nltk.tokenize.treebank import TreebankWordTokenizer
import nltk
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer

import random

import json
import sys

[nltk_data] Downloading package omw-1.4 to C:\Users\Lola Monroy
[nltk_data]     Mir\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Lola Monroy
[nltk_data]     Mir\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Lola Monroy
[nltk_data]     Mir\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Lola Monroy
[nltk_data]     Mir\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [103]:
with open('house_data.json') as f:
  data = json.load(f)

In [104]:
data['houses'].append({'id': 26, 'type': 'sale', 'bedrooms': '5', 'bathrooms': '3', 'price': '200k', 'square_meters': '90', 'floor': '6', 'elevator': 'Yes', 'commercial_use': 'Yes', 'terrace': 'Yes', 'location': 'Tarragona'})

id = 26
house = data['houses'][id-1]

In [105]:
def print_question(prompt, possible_options = []):
    print(prompt)  # El chatbot imprimeix la pregunta 
    if not len(possible_options) == 0:  # En cas que la llista d'opcions NO estigui buida -> ha d'oferir opcions
      print("Options:", ", ".join(possible_options)) # Ofereix a l'usuari les possibles respostes 

def initialize_available_options(house_data, available_options): # available_options és un dict 
    for house in house_data['houses']: 
        for key, value in house.items():  # Per cada característica de cada casa
            available_options.setdefault(key, set()).add(value)  # Guarda un set per cada característica amb totes les opcions que hi ha per cada una (un set per no tenir repetits)

def preprocess_answer(answer): # Tokenitza la resposta del usuari
    answer = nltk.word_tokenize(answer)
    return answer 

def get_numerical_value(tok_answer): # Extreu valors numèrics de la resposta tokenitzada amb la funció anterior
    for token in tok_answer:
        if token.isnumeric() or token[:-1].isnumeric(): # Mira si el token és numèric o si sense l'ultim char ho és (per treure 'k' de 35k)
            return token 
    return '' # Només retorna un valor numèric si el troba, sinó retorna '' osigui res

In [106]:
class QuitException(Exception):
    """Excepción personalizada para manejar cuando el usuario quiere salir."""
    pass

# TO DO: MODIFIQUEM LES FUNCIONS PERQUÈ ACCEPTI ANY 
def process_numerical_question(question):
    print_question(question['question']) 
    while True:
        answer = input(question['prompt'])  
        if str(answer).lower() == 'quit':
            raise QuitException(data['end_message'])
        elif answer.lower() == 'any':  
            return 'any'
        tok_answer = preprocess_answer(answer)
        value = get_numerical_value(tok_answer)
        if value != '': 
            return value

def process_multichoice_question(question, options):
    print_question(question['question'], options)  
    while True:
        answer = input(question['prompt'])
        if str(answer).lower() == 'quit':
            raise QuitException(data['end_message'])
        elif answer.lower() == 'any':  
            return 'any'
        if answer in options: 
            return answer
        
def convert_k_to_number(value):
    if isinstance(value, str) and 'k' in value.lower():
        return int(value.lower().replace('k', '000'))
    elif isinstance(value, str) and value.isdigit():
        return int(value)
    return value  

        
def find_suitable_houses(data, user_preferences):
    suitable_houses = []


    filters = {
        'bedrooms': lambda answer, house: int(house) >= int(answer) if answer != 'any' else True,
        'bathrooms': lambda answer, house: int(house) >= int(answer) if answer != 'any' else True,
        'price': lambda answer, house: convert_k_to_number(house) <= convert_k_to_number(answer) if answer != 'any' else True,
        'square_meters': lambda answer, house: int(house) >= int(answer) if answer != 'any' else True,
        'location': lambda answer, house: house == answer if answer != 'any' else True,
        'type': lambda answer, house: house == answer if answer != 'any' else True,
        'floor': lambda answer, house: int(house) >= int(answer) if answer != 'any' else True,
        'terrace': lambda answer, house: house == 'Yes' if answer == 'Yes' else True,  
        'elevator': lambda answer, house: house == 'Yes' if answer == 'Yes' else True,  
        'commercial_use': lambda answer, house: house == 'Yes' if answer == 'Yes' else True  
        }
    

    for house in data['houses']:
        is_suitable = True
        for key, filter_func in filters.items():
            if key in user_preferences:
                if not filter_func(user_preferences[key], house[key]):
                    is_suitable = False
                    break
        if is_suitable:
            suitable_houses.append(house)
    
    return suitable_houses

In [107]:
start_message = random.choice(data['start_messages'])
print(start_message)

user_preferences, available_options = {}, {} 

initialize_available_options(data, available_options) 

for question in data['questions']:
  answer_key = question['answer_key'] 
  possible_options = list(available_options.get(answer_key, []))
  
  if question['type'] == 'numerical':
    answer = process_numerical_question(question)
  else:
    answer = process_multichoice_question(question, possible_options)

  user_preferences[answer_key] = answer 

def print_suitable_houses(suitable_houses):
  if suitable_houses:
    print("\nBased on your preferences, the most suitable houses are:")
    print()
    for house in suitable_houses:
      # Text 'humanitzat': faltaria corregir el ús de sigulars y plurals
      print(f"House with ID", house['id'], "is currently for", house['type'],". It has", house['bedrooms'], "bedrooms and", house['bathrooms'], "bathrooms.")
      print("Its price is of exactly", house['price'], "euros and it has", house['square_meters'], "m^2.")
      print("It is located in", house['location'])
      print()
  else:
    print("\nSorry, no suitable houses match your preferences. \n")

suitable_houses = find_suitable_houses(data, user_preferences)
print_suitable_houses(suitable_houses)

end_message = random.choice(data['end_messages'])
print(end_message)

Welcome to the House Buying Assistant!
How many bedrooms do you need?
How many bathrooms do you need?
What is your budget for the house?
How many square meters do you need?
Which city or neighborhood would you prefer?
Options: Tarragona, L'Hospitalet de Llobregat, Esplugues de Llobregat, Santa Coloma de Gramenet, Barcelona
Are you looking for house on sale or on rent?
Options: rent, sale
What floor do you wish to live in?
Do you wish to have a terrace?
Options: No, Yes
Do you wish to have an elevator?
Options: No, Yes
Do you wish to use it for commercial use

Based on your preferences, the most suitable houses are:

House with ID 1 is currently for rent . It has 3 bedrooms and 2 bathrooms.
Its price is of exactly 1200 euros and it has 100 m^2.
It is located in Santa Coloma de Gramenet

House with ID 2 is currently for sale . It has 4 bedrooms and 2 bathrooms.
Its price is of exactly 250k euros and it has 120 m^2.
It is located in Santa Coloma de Gramenet

House with ID 3 is currently f